In [11]:
%matplotlib inline
import numpy as np
import os

import matplotlib.pyplot as plt
import time, cv2
import seaborn as sns

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import model_from_json

import const

In [12]:
#Création de datagenerator pour modifier nos images au fil des epochs
train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input,
                                          rescale=1.0 / 255, #Normalisation
                                          featurewise_center=True, #Soustraction de la valeur moyenne de chaque pixel
                                          featurewise_std_normalization=True,
                                          rotation_range = 30,
                                          horizontal_flip = True)
valid_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


In [13]:
#Creation des iterators sur les images depuis les dossiers
train_path = os.path.join(const.DATASET_CLEAN_WO_BACKGROUND, "train")
valid_path = os.path.join(const.DATASET_CLEAN_WO_BACKGROUND, "valid")
test_path = os.path.join(const.DATASET_CLEAN_WO_BACKGROUND, "test")
batch_size = 16
train_generator = train_data_generator.flow_from_directory(directory=train_path,
                                                           class_mode = "sparse",
                                                           target_size=(224,224),
                                                           batch_size=batch_size)
valid_generator = valid_data_generator.flow_from_directory(directory=valid_path,
                                                           class_mode = "sparse",
                                                           target_size=(224,224),
                                                           batch_size=batch_size)
test_generator = test_data_generator.flow_from_directory(directory=test_path,
                                                           class_mode = "sparse",
                                                           target_size=(224,224),
                                                           batch_size=batch_size)

Found 46958 images belonging to 443 classes.
Found 9746 images belonging to 443 classes.
Found 9746 images belonging to 443 classes.


In [14]:
def create_model_VGG16(n_class, nb_defroze):
    #Creation du model
    model = Sequential()
    base_model = VGG16(weights='imagenet', include_top=False) 
    n_class = 443
    # Freezer les couches du VGG16
    for layer in base_model.layers[:len(base_model.layers) - nb_defroze]: 
        layer.trainable = False

    model.add(base_model) # Ajout du modèle VGG16
    model.add(GlobalAveragePooling2D()) 
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(n_class, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])
    return model

def run_model(model, train_generator, valid_generator, model_name):
    nb_image_train = train_generator.samples
    nb_image_valid = valid_generator.samples
    history = model.fit(train_generator,
                              epochs=8, 
                              steps_per_epoch=nb_image_train//batch_size,
                              validation_data=valid_generator, 
                              validation_steps=nb_image_valid//batch_size)
    model_json = model.to_json()
    with open(model_name + ".json", "w") as file:
        file.write(model_json)

    # serialize weights to HDF5
    model.save_weights(model_name + ".h5")
    print("Saved model to disk")

    test_accuracy = model.evaluate(test_generator)
    print(f"Test accuracy: {test_accuracy:.2f}")
    print(test_accuracy)
    return history

In [ ]:
model_model_wo_defroze = create_model_VGG16(n_class=443, nb_defroze=0)
history_model_wo_defroze = run_model(model_model_wo_defroze, train_generator, valid_generator, "model_wo_defroze")

In [ ]:
model_model_4_defroze = create_model_VGG16(n_class=443, nb_defroze=-4)
history_model_4_defroze = run_model(model_model_4_defroze, train_generator, valid_generator, "model_4_defroze")

In [ ]:
score = model_model_4_defroze.evaluate(test_generator)
print("%s: %.2f%%" % (model_model_4_defroze.metrics_names[1], score[1]*100))

In [49]:
# pred_labels = model.predict(test_generator)
true_labels = test_generator.classes
predicted_classes = np.argmax(pred_labels, axis=1)

print(classification_report(true_labels, predicted_classes))



              precision    recall  f1-score   support

           0       0.36      0.38      0.37        21
           1       0.00      0.00      0.00        21
           2       0.14      0.14      0.14        21
           3       0.24      0.33      0.28        21
           4       0.23      0.33      0.27        21

    accuracy                           0.24       105
   macro avg       0.19      0.24      0.21       105
weighted avg       0.19      0.24      0.21       105



In [11]:
model_json = model_model_4_defroze.to_json()
with open("model_model_4_defroze.json", "w") as file:
    file.write(model_json)

# serialize weights to HDF5
model_model_4_defroze.save_weights("model_model_4_defroze.h5")
print("Saved model to disk")

Saved model to disk


In [12]:
# load json and create model
with open('model_model_4_defroze.json', 'r') as json_file:
    loaded_model_json = json_file.read()
model_model_4_defroze = model_from_json(loaded_model_json)
# load weights into new model
model_model_4_defroze.load_weights("model_model_4_defroze.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
model_model_4_defroze.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])


Loaded model from disk
